# Paper Plotter

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

import torch

In [ ]:
import sys
sys.path.append('../src')

In [ ]:
from systems.LJ import lennard_jones

In [ ]:
device = torch.device("cuda:0")

dimensions = 2
n_particles = 32
cutin = 0.8
dofs = n_particles*dimensions

T_source = 2
beta_source = 1/T_source
box_length_source = 6.6
rho_source = n_particles/(box_length_source)**(dimensions)
WCA = lennard_jones(n_particles=n_particles, dimensions=dimensions, rho=rho_source, device=device, cutin=cutin, cutoff="wca")
box_length_pr = WCA.box_length

# rho_target = 0.70408163
# T_target = 0.60816327
# beta_target = 1/T_target
box_length_target = 6.6
rho_target = n_particles/(box_length_target)**(dimensions)
T_target = 1
beta_target = 1/T_target
scale = (rho_source/rho_target)**(1/dimensions)
LJ = lennard_jones(n_particles=n_particles, dimensions=dimensions, rho=rho_target, device=device, cutin=cutin)
box_length_sys = LJ.box_length
box_length_target = box_length_sys[0].item()

print(f"rho_source = {rho_source}, T_source = {T_source}")
print(f"rho_target = {rho_target}, T_target = {T_target}")
print(f"s = {scale}")

In [ ]:
run_id = f"NVT_N{n_particles:03d}_WCA2LJ_rho_{rho_source:.2g}_T{T_source:.2g}_to_rho_{rho_target:.2g}_T{T_target:.2g}_main"

output_dir = os.path.join("./output", run_id)
gendir = os.path.join(output_dir, "generated_confs")
assert os.path.exists(gendir), "Folder with generated configurations not found!"

In [ ]:
wca_train_filepath = f"./data/N{WCA.n_particles:03d}/{WCA.name}/rho_{rho_source:.02g}_T_{T_source:.02g}_train.pt"
wca_sample_filepath = f"./data/N{WCA.n_particles:03d}/{WCA.name}/rho_{rho_source:.02g}_T_{T_source:.02g}_sample.pt"

print()
print("Loading WCA Training Datasets")
wca_train = torch.load(wca_train_filepath, map_location=device)
print(f"WCA Train Dataset: {wca_train_filepath}")
wca_sample = torch.load(wca_sample_filepath, map_location=device)
print(f"WCA Sample Dataset: {wca_sample_filepath}")

lj_train_filepath = f"./data/N{LJ.n_particles:03d}/{LJ.name}/rho_{rho_target:.02g}_T_{T_target:.02g}_train.pt"
lj_sample_filepath = f"./data/N{LJ.n_particles:03d}/{LJ.name}/rho_{rho_target:.02g}_T_{T_target:.02g}_sample.pt"

print()
print("Loading LJ Training Datasets")
lj_train = torch.load(lj_train_filepath, map_location=device)
print(f"LJ Train Dataset: {lj_train_filepath}")
lj_sample = torch.load(lj_sample_filepath, map_location=device)
print(f"LJ Sample Dataset: {lj_sample_filepath}")

wca_train_cpu = wca_train.view(-1, n_particles, dimensions).cpu().numpy()
wca_sample_cpu = wca_sample.view(-1, n_particles, dimensions).cpu().numpy()
lj_train_cpu = lj_train.view(-1, n_particles, dimensions).cpu().numpy()
lj_sample_cpu = lj_sample.view(-1, n_particles, dimensions).cpu().numpy()

wca_energy_train_cpu = WCA.energy(wca_train).squeeze().cpu().numpy()
lj_energy_train_cpu = LJ.energy(lj_train).squeeze().cpu().numpy()
wca_energy_sample_cpu = WCA.energy(wca_sample).squeeze().cpu().numpy()
lj_energy_sample_cpu = LJ.energy(lj_sample).squeeze().cpu().numpy()

print()
print(f"Prior train size: {wca_train.shape[0]}")
print(f"Prior sample size: {wca_sample.shape[0]}")
print(f"Posterior train size: {lj_train.shape[0]}")
print(f"Posterior sample size: {lj_sample.shape[0]}")

# Z --> X

## Load replicas

In [ ]:
import torch

N_samples = 100000
N_batch_samples_stride = 10
N_batch_samples_size = N_samples//N_batch_samples_stride

NN_generated_samples = 500000
N_replicas = 10
NN_replica_size = NN_generated_samples//N_replicas

NN_batch_size = 50000
if NN_batch_size > NN_replica_size:
    NN_batch_size = NN_replica_size
NN_batch_stride = NN_replica_size//NN_batch_size

print("N_samples = {}".format(N_samples))
print("N_batch_samples_stride = {}".format(N_batch_samples_stride))
print("N_batch_samples_size = {}".format(N_batch_samples_size))
print()
print("NN_generated_samples = {}".format(NN_generated_samples))
print("N_replicas = {}".format(N_replicas))
print("NN_replica_size = {}".format(NN_replica_size))
print()
print("NN_batch_size = {}".format(NN_batch_size))
print("NN_batch_stride = {}".format(NN_batch_stride))
print()

z = torch.zeros((N_replicas, NN_replica_size, dofs), device=device)
Tz = torch.zeros((N_replicas, NN_replica_size, dofs), device=device)
log_w_zx = torch.zeros((N_replicas, NN_replica_size), device=device)
Tz_resampled = torch.zeros((N_replicas, NN_replica_size, dofs), device=device)

In [ ]:
from tqdm import tqdm

print("Loading generated configurations prior to system... ", flush=True, end="")
for r in tqdm(range(N_replicas), ncols=100, desc="\tProgress"):
    z_generated_filepath = os.path.join(gendir, f"z_{r:04d}.pt")
    z[r] = torch.load(z_generated_filepath, map_location=device)
    Tz_generated_filepath = os.path.join(gendir, f"Tz_{r:04d}.pt")
    Tz[r] = torch.load(Tz_generated_filepath, map_location=device)
    log_w_zx_generated_filepath = os.path.join(gendir, f"log_w_zx_{r:04d}.pt")
    log_w_zx[r] = torch.load(log_w_zx_generated_filepath)
    Tz_resampled_generated_filepath = os.path.join(gendir, f"Tz_resampled_{r:04d}.pt")
    Tz_resampled[r] = torch.load(Tz_resampled_generated_filepath, map_location=device)
print("Done")
print()

## Energies

In [ ]:
# Redefining energy function

def U_x(x):

    return LJ.energy(x).squeeze(axis=-1).cpu().numpy()


In [ ]:
def bootstrap_observable(observable_fn, samples, n_bootstrap, ref_bins, log_weights=None, batch_size=None):

    N = samples.shape[0]
    if batch_size is None:
        batch_size = N
    n_batches = N//batch_size

    U_batch = np.zeros((n_batches, batch_size))
    U_bstrp = np.zeros((n_bootstrap, N))
    U_hist = np.zeros((n_bootstrap, ref_bins.shape[0]-1))
    
    for n in tqdm(range(n_bootstrap), ncols=100, desc="\tProgress"):

        indx = np.random.choice(N, size=N, replace=True, p=None)

        for batch in range(n_batches):
            batch_indx = indx[batch*batch_size:(batch+1)*batch_size]
            U_batch[batch] = observable_fn(samples[batch_indx])
        U_bstrp[n] = U_batch.ravel()

        if log_weights is not None:
            weights = np.exp(log_weights[indx] - log_weights[indx].max())
        else:
            weights = None

        U_hist[n], _ = np.histogram(U_bstrp[n], bins=ref_bins, density=True, weights=weights)
        
    return U_hist.mean(axis=0), (ref_bins[1:] + ref_bins[:-1])/2, U_hist.std(axis=0, ddof=1)/np.sqrt(n_bootstrap)


In [ ]:
# Energies calculation
U_zx_reference_batch = np.zeros((N_batch_samples_stride, N_batch_samples_size))
U_zx_identity_batch = np.zeros((NN_batch_stride, NN_batch_size))
U_zx_transformed_batch = np.zeros((NN_batch_stride, NN_batch_size))
U_zx_resampled_batch = np.zeros((NN_batch_stride, NN_batch_size))

n_bins = 80
n_bootstrap = 100
print("n_bins = {}".format(n_bins))
print("n_bootstrap = {}".format(n_bootstrap))
print()

bin_edges_reference_batch = np.zeros((N_batch_samples_stride, n_bins+1))
bin_edges_identity = np.zeros((N_replicas, n_bins+1))
bin_edges_identity_batch = np.zeros((NN_batch_stride, n_bins+1))
bin_edges_transformed = np.zeros((N_replicas, n_bins+1))
bin_edges_transformed_batch = np.zeros((NN_batch_stride, n_bins+1))
bin_edges_resampled = np.zeros((N_replicas, n_bins+1))
bin_edges_resampled_batch = np.zeros((NN_batch_stride, n_bins+1))
U_zx_reference_hist_batch = np.zeros((N_batch_samples_stride, n_bins))
U_zx_identity_hist = np.zeros((N_replicas, n_bins))
U_zx_identity_hist_batch = np.zeros((NN_batch_stride, n_bins))
U_zx_transformed_hist = np.zeros((N_replicas, n_bins))
U_zx_transformed_hist_batch = np.zeros((NN_batch_stride, n_bins))
U_zx_resampled_hist = np.zeros((N_replicas, n_bins))
U_zx_resampled_hist_batch = np.zeros((NN_batch_stride, n_bins))
U_zx_bootstrap = np.zeros((N_replicas, n_bins))
Delta_U_zx_bootstrap = np.zeros((N_replicas, n_bins))

for batch_start in range(N_batch_samples_stride):
    U_zx_reference_batch[batch_start] = U_x(lj_train[batch_start::N_batch_samples_stride])

U_zx_ref_min, U_zx_ref_max = np.min(U_zx_reference_batch), np.max(U_zx_reference_batch) 

U_zx_id_min, U_zx_id_max = np.inf, -np.inf
U_zx_res_min, U_zx_res_max = np.inf, -np.inf
for r in range(N_replicas):
    for batch_start in range(NN_batch_stride):
        U_zx_identity_batch[batch_start] = U_x(scale*z[r, batch_start::NN_batch_stride])
        U_zx_transformed_batch[batch_start] = U_x(Tz[r, batch_start::NN_batch_stride])
        U_zx_resampled_batch[batch_start] = U_x(Tz_resampled[r, batch_start::NN_batch_stride])
    U_zx_id_batch_min, U_zx_id_batch_max = np.min(U_zx_identity_batch), np.max(U_zx_identity_batch) 
    U_zx_res_batch_min, U_zx_res_batch_max = np.min(U_zx_resampled_batch), np.max(U_zx_resampled_batch) 
U_zx_id_min, U_zx_id_max = np.min([U_zx_id_min, U_zx_id_batch_min]), np.max([U_zx_id_max, U_zx_id_batch_max]) 
U_zx_res_min, U_zx_res_max = np.min([U_zx_res_min, U_zx_res_batch_min]), np.max([U_zx_res_max, U_zx_res_batch_max]) 

ref_bins = np.linspace(np.min( [U_zx_ref_min, U_zx_id_min, U_zx_res_min]),
                       np.max( [U_zx_ref_max, U_zx_id_max, U_zx_res_max]),
                       n_bins+1)
bin_width = (ref_bins[1]-ref_bins[0])
mids = (ref_bins[1:] + ref_bins[:-1])/2

# Prior to System
for batch_start in range(N_batch_samples_stride):
    U_zx_reference_hist_batch[batch_start], bin_edges_reference_batch[batch_start] = np.histogram(U_zx_reference_batch[batch_start], bins=ref_bins, density=True)
U_zx_reference_hist, bin_edges_reference = U_zx_reference_hist_batch.mean(axis=0), bin_edges_reference_batch.mean(0)

for r in range(N_replicas):
    for batch_start in range(NN_batch_stride):
        U_zx_identity_batch[batch_start] = U_x(scale*z[r, batch_start::NN_batch_stride])
        U_zx_identity_hist_batch[batch_start], bin_edges_identity_batch[batch_start] = np.histogram(U_zx_identity_batch[batch_start], bins=ref_bins, density=True)
        U_zx_transformed_batch[batch_start] = U_x(Tz[r, batch_start::NN_batch_stride])
        U_zx_transformed_hist_batch[batch_start], bin_edges_transformed_batch[batch_start] = np.histogram(U_zx_transformed_batch[batch_start], bins=ref_bins, density=True)
        U_zx_resampled_batch[batch_start] = U_x(Tz_resampled[r, batch_start::NN_batch_stride])
        U_zx_resampled_hist_batch[batch_start], bin_edges_resampled_batch[batch_start] = np.histogram(U_zx_resampled_batch[batch_start], bins=ref_bins, density=True)
    U_zx_identity_hist[r], bin_edges_identity[r] = U_zx_identity_hist_batch.mean(axis=0), bin_edges_identity_batch.mean(axis=0)
    U_zx_transformed_hist[r], bin_edges_transformed[r] = U_zx_transformed_hist_batch.mean(axis=0), bin_edges_transformed_batch.mean(axis=0)
    U_zx_resampled_hist[r], bin_edges_resampled[r] = U_zx_resampled_hist_batch.mean(axis=0), bin_edges_resampled_batch.mean(axis=0)

for r in range(N_replicas):
    U_zx_bootstrap[r], _, Delta_U_zx_bootstrap[r] = bootstrap_observable(U_x, Tz[r], n_bootstrap, ref_bins, log_w_zx[r].cpu().numpy(), batch_size=None)


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

fig_size = (10 * 0.393701, 6 * 0.393701)
fig = plt.figure(figsize = fig_size, dpi = 400)
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('text.latex', preamble=r"\usepackage{braket}")

plt.bar(mids, U_zx_reference_hist, alpha=0.5, width=bin_width, label=r"Reference", color="C0")
for r in range(N_replicas):
    if r == 0:
        plt.bar(mids, U_zx_resampled_hist[r], alpha=0.2, width=bin_width, label=r"Resampled", color="C3")
    else:
        plt.bar(mids, U_zx_resampled_hist[r], alpha=0.2, width=bin_width, color="C3")

plt.legend(frameon=False)

plt.xlabel(r"Energy(RU)")
plt.title(r"Energy $\mathrm{A} \to \mathrm{B}$")
plt.show()

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

fig_size = (10 * 0.393701, 6 * 0.393701)
fig = plt.figure(figsize = fig_size, dpi = 400)
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('text.latex', preamble=r"\usepackage{braket}")

plt.bar(mids, U_zx_reference_hist, alpha=0.5, width=bin_width, label=r"Reference", color="C0")
for r in range(N_replicas):
    if r == 0:
        plt.bar(mids, U_zx_bootstrap[r], alpha=0.2, width=bin_width, label=r"Bootstrapped", color="C3")
    else:
        plt.bar(mids, U_zx_bootstrap[r], alpha=0.2, width=bin_width, color="C3")

plt.legend(frameon=False)

plt.xlabel(r"Energy(RU)")
plt.title(r"Energy $\mathrm{A} \to \mathrm{B}$")
plt.show()

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

fig_size = (10 * 0.393701, 6 * 0.393701)
fig = plt.figure(figsize = fig_size, dpi = 400)
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('text.latex', preamble=r"\usepackage{braket}")

plt.bar(mids, U_zx_reference_hist, alpha=0.5, width=bin_width, label=r"Reference", color="C0", zorder=25)
plt.bar(mids, U_zx_identity_hist.mean(axis=0), alpha=0.5, width=bin_width, label=r"Identity", color="C1", zorder=25)
plt.bar(mids, U_zx_transformed_hist.mean(axis=0), alpha=0.5, width=bin_width, label=r"Transformed", color="C2", zorder=25)
Delta_U_zx_bootstrap = U_zx_bootstrap.std(axis=0, ddof=1)/np.sqrt(N_replicas)
plt.bar(mids, U_zx_bootstrap.mean(axis=0), color="C3", alpha=0.5, width=bin_width, yerr=Delta_U_zx_bootstrap, error_kw={'ecolor' : 'C3', 'alpha' : 0., 'lw' : 2.6}, label=r"Reweighted", zorder=25)
# Delta_U_zx_resampled = U_zx_resampled_hist.std(axis=0, ddof=1)/np.sqrt(N_replicas)
# plt.bar(mids, U_zx_resampled_hist.mean(axis=0), color="C4", alpha=0.5, width=bin_width, yerr=Delta_U_zx_resampled, error_kw={'ecolor' : 'C4', 'alpha' : 0, 'lw' : 2.6}, label=r"Resampled", zorder=25)

U_mean_reference = np.average(mids, weights=U_zx_reference_hist)
U_mean_bootstrap = np.average(mids, weights=U_zx_bootstrap.mean(axis=0))
U_mean_resampled = np.average(mids, weights=U_zx_resampled_hist.mean(axis=0))
plt.axvline(x = U_mean_reference, ymin = 0, c='C0', ls="--", lw=1, alpha=0.5, label=r"$\braket{U_\mathrm{B}}$ Reference")
plt.axvline(x = U_mean_bootstrap, ymin = 0, c='C3', ls="--", lw=1, alpha=0.5, label=r"$\Braket{U_\mathrm{B}\bigl(F(x_{\mathrm{A}})\bigr)}$")
# plt.axvline(x = U_mean_resampled, ymin = 0, c='C4', ls="--", lw=1, alpha=0.5, label=r"$\Braket{U_\mathrm{B}\bigl(F(x_{\mathrm{A}})\bigr)}$")

xlim = plt.xlim()
ylim = plt.ylim()

# plt.bar(U_mean_reference, height=ylim[1]+1, width=1, alpha=0.25, color='C0', zorder=0)
# plt.bar(U_mean_bootstrap, height=ylim[1]+1, width=1, alpha=0.25, color='C3', zorder=5)
# plt.bar(U_mean_resampled, height=ylim[1]+1, width=1, alpha=0.25, color='C4', zorder=5)

#get handles and labels
handles, labels = plt.gca().get_legend_handles_labels()

#specify order of items in legend
order = [2,3,4,5,0,1]

#add legend to plot
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order],frameon=False) 

# x = []
# y = []
# xlabels = []
# ylabels = []
# plt.xticks(x, xlabels)
# plt.yticks(y, ylabels)

# plt.legend(frameon=False) 

plt.xlim(xlim)
plt.ylim(0, ylim[1])
plt.xlabel(r"$U^*$")
# plt.title(r"Potential Energy Distribution $\mathrm{A} \to \mathrm{B}$")
plt.savefig(os.path.join(gendir, "energyA2B.png"))
plt.show()

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

fig_size = (10 * 0.393701, 6 * 0.393701)
fig = plt.figure(figsize = fig_size, dpi = 400)
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('text.latex', preamble=r"\usepackage{braket}")

linew = 0.2
bck_alpha = 0.4

plt.plot(mids, U_zx_reference_hist, label=r"Reference", lw=linew, color="C0")
plt.fill_between(mids, np.zeros_like(U_zx_reference_hist), U_zx_reference_hist, color="C0", alpha=bck_alpha)

plt.plot(mids, U_zx_identity_hist.mean(0), label=r"Untransformed", lw=linew, color="C1")
plt.fill_between(mids, np.zeros_like(U_zx_identity_hist.mean(0)), U_zx_identity_hist.mean(0), color="C1", alpha=bck_alpha)

plt.plot(mids, U_zx_transformed_hist.mean(0), label=r"Transformed", lw=linew, color="C2")
plt.fill_between(mids, np.zeros_like(U_zx_transformed_hist.mean(0)), U_zx_transformed_hist.mean(0), color="C2", alpha=bck_alpha)

xlim = plt.xlim()
ylim = plt.ylim()

density_mean = np.average(U_zx_resampled_hist, axis=0)
density_std = np.std(U_zx_resampled_hist, axis=0)
plt.plot(mids, density_mean, label=r"Resampled", lw=linew, color="C3")
plt.fill_between(mids, density_mean-density_std, density_mean+density_std, label=r"Resampled", lw=0, color="C3", alpha=0.5)
plt.fill_between(mids, np.zeros_like(density_mean), density_mean, color="C3", alpha=bck_alpha)

# xlim = plt.xlim()
plt.ylim(ylim)
plt.xlabel(r"Energy (RU)")
plt.title(r"Potential Energy Distribution $\mathrm{A} \to \mathrm{B}$")
# plt.savefig(results_filepath)
plt.show()

## Radial Distribution Functions

In [ ]:
from tools.observables import rdf


n_bins = 100
cutoff = box_length_target/2
# Prior to System
RDF_r, RDF_zx_reference = rdf(lj_sample, n_particles=n_particles, dimensions=dimensions, box_length=box_length_sys, cutoff=cutoff, n_bins=n_bins, batch_size=None)

# RDF calculation
RDF_zx_identity = np.zeros((N_replicas, n_bins))
RDF_zx_transformed = np.zeros((N_replicas, n_bins))
RDF_zx_resampled = np.zeros((N_replicas, n_bins))
RDF_zx_reweighted = np.zeros((N_replicas, n_bins))

for r in range(N_replicas):
    _, RDF_zx_identity[r] = rdf(scale*z[r], n_particles=n_particles, dimensions=dimensions, box_length=box_length_sys, cutoff=cutoff, n_bins=n_bins, batch_size=None)     
    _, RDF_zx_transformed[r] = rdf(Tz[r], n_particles=n_particles, dimensions=dimensions, box_length=box_length_sys, cutoff=cutoff, n_bins=n_bins, batch_size=None)
    _, RDF_zx_resampled[r] = rdf(Tz_resampled[r], n_particles=n_particles, dimensions=dimensions, box_length=box_length_sys, cutoff=cutoff, n_bins=n_bins, batch_size=None)
    _, RDF_zx_reweighted[r] = rdf(Tz[r], n_particles=n_particles, dimensions=dimensions, box_length=box_length_sys, cutoff=cutoff, n_bins=n_bins, batch_size=None, log_weights=log_w_zx[r])


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

fig_size = (10 * 0.393701, 6 * 0.393701)
fig = plt.figure(figsize = fig_size, dpi = 400)
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('text.latex', preamble=r"\usepackage{braket}")

plt.axhline(y = 1, c='k', ls="--", lw=1, alpha=0.25)

plt.plot(RDF_r, RDF_zx_identity.mean(0), label=r"Identity", ls='--', color='C1')
plt.plot(RDF_r, RDF_zx_transformed.mean(0), label=r"Transformed", ls='-.', color='C2')
# plt.plot(RDF_r, RDF_zx_resampled.mean(axis=0), label=r"Resampled", ls='-', color='C3')
plt.plot(RDF_r, RDF_zx_reweighted.mean(axis=0), label=r"Reweighted", ls='-', color='C3')
plt.plot(RDF_r, RDF_zx_reference, label=r"Reference", ls=':', color='C0')

#get handles and labels
handles, labels = plt.gca().get_legend_handles_labels()

#specify order of items in legend
order = [3,0,1,2]

#add legend to plot
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order],frameon=False) 

# plt.legend(frameon=False)

plt.xlabel(r"$r^*$")
plt.ylabel(r"$g(r^*)$")
# plt.title(r"Radial Distribution Function $\mathrm{A}\to \mathrm{B}$")
plt.savefig(os.path.join(gendir, "rdfA2B.png"))
plt.show()

## RESS

In [ ]:
rep_zx, ress_id_zx, ress_zx = np.loadtxt(os.path.join(gendir, "ress_zx.out"), unpack=True, usecols=(0,1,2))
n_rep = rep_zx.shape[0]

In [ ]:
print("z->x")
print(f"ress id :{ress_id_zx.mean()} +- {ress_id_zx.std(ddof=1)/np.sqrt(n_rep)}\tDelta = {ress_id_zx.std(ddof=1)/np.sqrt(n_rep)/ress_id_zx.mean()}")
print(f"ress :{ress_zx.mean()} +- {ress_zx.std(ddof=1)/np.sqrt(n_rep)}\tDelta = {ress_zx.std(ddof=1)/np.sqrt(n_rep)/ress_zx.mean()}")
print(f"ratio : {ress_zx.mean()/ress_id_zx.mean()}")
print()

# X --> Z

## Loading Replicas

In [ ]:
import torch

N_samples = 100000
N_batch_samples_stride = 10
N_batch_samples_size = N_samples//N_batch_samples_stride

NN_generated_samples = 500000
N_replicas = 10
NN_replica_size = NN_generated_samples//N_replicas

NN_batch_size = 50000
if NN_batch_size > NN_replica_size:
    NN_batch_size = NN_replica_size
NN_batch_stride = NN_replica_size//NN_batch_size

print("N_samples = {}".format(N_samples))
print("N_batch_samples_stride = {}".format(N_batch_samples_stride))
print("N_batch_samples_size = {}".format(N_batch_samples_size))
print()
print("NN_generated_samples = {}".format(NN_generated_samples))
print("N_replicas = {}".format(N_replicas))
print("NN_replica_size = {}".format(NN_replica_size))
print()
print("NN_batch_size = {}".format(NN_batch_size))
print("NN_batch_stride = {}".format(NN_batch_stride))
print()

x = torch.zeros((N_replicas, NN_replica_size, dofs), device=device)
Tinvx = torch.zeros((N_replicas, NN_replica_size, dofs), device=device)
log_w_xz = torch.zeros((N_replicas, NN_replica_size), device=device)
Tinvx_resampled = torch.zeros((N_replicas, NN_replica_size, dofs), device=device)


In [ ]:
from tqdm import tqdm

print("Loading generated configurations system to prior... ", flush=True, end="")
for r in tqdm(range(N_replicas), ncols=100, desc="\tProgress"):
    x_generated_filepath = os.path.join(gendir, f"x_{r:04d}.pt")
    x[r] = torch.load(x_generated_filepath)
    Tinvx_generated_filepath = os.path.join(gendir, f"Tinvx_{r:04d}.pt")
    Tinvx[r] = torch.load(Tinvx_generated_filepath)
    log_w_xz_generated_filepath = os.path.join(gendir, f"log_w_xz_{r:04d}.pt")
    log_w_xz[r] = torch.load(log_w_xz_generated_filepath)
    Tinvx_resampled_generated_filepath = os.path.join(gendir, f"Tinvx_resampled_{r:04d}.pt")
    Tinvx_resampled[r] = torch.load(Tinvx_resampled_generated_filepath)
print("Done")
print()

## Energies

In [ ]:
# Redefining energy function

def U_z(z):

    return WCA.energy(z).squeeze(axis=-1).cpu().numpy()


In [ ]:
# Energies calculation
U_xz_reference_batch = np.zeros((N_batch_samples_stride, N_batch_samples_size))
U_xz_identity_batch = np.zeros((NN_batch_stride, NN_batch_size))
U_xz_transformed_batch = np.zeros((NN_batch_stride, NN_batch_size))
U_xz_resampled_batch = np.zeros((NN_batch_stride, NN_batch_size))

n_bins = 80
n_bootstrap = 100
print("n_bins = {}".format(n_bins))
print("n_bootstrap = {}".format(n_bootstrap))
print()

bin_edges_reference_batch = np.zeros((N_batch_samples_stride, n_bins+1))
bin_edges_identity = np.zeros((N_replicas, n_bins+1))
bin_edges_identity_batch = np.zeros((NN_batch_stride, n_bins+1))
bin_edges_transformed = np.zeros((N_replicas, n_bins+1))
bin_edges_transformed_batch = np.zeros((NN_batch_stride, n_bins+1))
bin_edges_resampled = np.zeros((N_replicas, n_bins+1))
bin_edges_resampled_batch = np.zeros((NN_batch_stride, n_bins+1))
U_xz_reference_hist_batch = np.zeros((N_batch_samples_stride, n_bins))
U_xz_identity_hist = np.zeros((N_replicas, n_bins))
U_xz_identity_hist_batch = np.zeros((NN_batch_stride, n_bins))
U_xz_transformed_hist = np.zeros((N_replicas, n_bins))
U_xz_transformed_hist_batch = np.zeros((NN_batch_stride, n_bins))
U_xz_resampled_hist = np.zeros((N_replicas, n_bins))
U_xz_resampled_hist_batch = np.zeros((NN_batch_stride, n_bins))
U_xz_bootstrap = np.zeros((N_replicas, n_bins))
Delta_U_xz_bootstrap = np.zeros((N_replicas, n_bins))

for batch_start in range(N_batch_samples_stride):
    U_xz_reference_batch[batch_start] = U_z(wca_train[batch_start::N_batch_samples_stride])

U_xz_ref_min, U_xz_ref_max = np.min(U_xz_reference_batch), np.max(U_xz_reference_batch) 

U_xz_id_min, U_xz_id_max = np.inf, -np.inf
U_xz_res_min, U_xz_res_max = np.inf, -np.inf
for r in range(N_replicas):
    for batch_start in range(NN_batch_stride):
        U_xz_identity_batch[batch_start] = U_z(x[r, batch_start::NN_batch_stride]/scale)
        U_xz_transformed_batch[batch_start] = U_z(Tinvx[r, batch_start::NN_batch_stride])
        U_xz_resampled_batch[batch_start] = U_z(Tinvx_resampled[r, batch_start::NN_batch_stride])
    U_xz_id_batch_min, U_xz_id_batch_max = np.min(U_xz_identity_batch), np.max(U_xz_identity_batch) 
    U_xz_res_batch_min, U_xz_res_batch_max = np.min(U_xz_resampled_batch), np.max(U_xz_resampled_batch) 
U_xz_id_min, U_xz_id_max = np.min([U_xz_id_min, U_xz_id_batch_min]), np.max([U_xz_id_max, U_xz_id_batch_max]) 
U_xz_res_min, U_xz_res_max = np.min([U_xz_res_min, U_xz_res_batch_min]), np.max([U_xz_res_max, U_xz_res_batch_max]) 

ref_bins = np.linspace(np.min( [U_xz_ref_min, U_xz_id_min, U_xz_res_min]),
                   np.max( [U_xz_ref_max, U_xz_id_max, U_xz_res_max]),
                   n_bins+1)
bin_width = (ref_bins[1]-ref_bins[0])
mids = (ref_bins[1:] + ref_bins[:-1])/2

# Prior to System
for batch_start in range(N_batch_samples_stride):
    U_xz_reference_hist_batch[batch_start], bin_edges_reference_batch[batch_start] = np.histogram(U_xz_reference_batch[batch_start], bins=ref_bins, density=True)
U_xz_reference_hist, bin_edges_reference = U_xz_reference_hist_batch.mean(axis=0), bin_edges_reference_batch.mean(0)

for r in range(N_replicas):
    for batch_start in range(NN_batch_stride):
        U_xz_identity_batch[batch_start] = U_z(x[r, batch_start::NN_batch_stride]/scale)
        U_xz_identity_hist_batch[batch_start], bin_edges_identity_batch[batch_start] = np.histogram(U_xz_identity_batch[batch_start], bins=ref_bins, density=True)
        U_xz_transformed_batch[batch_start] = U_z(Tinvx[r, batch_start::NN_batch_stride])
        U_xz_transformed_hist_batch[batch_start], bin_edges_transformed_batch[batch_start] = np.histogram(U_xz_transformed_batch[batch_start], bins=ref_bins, density=True)
        U_xz_resampled_batch[batch_start] = U_z(Tinvx_resampled[r, batch_start::NN_batch_stride])
        U_xz_resampled_hist_batch[batch_start], bin_edges_resampled_batch[batch_start] = np.histogram(U_xz_resampled_batch[batch_start], bins=ref_bins, density=True)
    U_xz_identity_hist[r], bin_edges_identity[r] = U_xz_identity_hist_batch.mean(axis=0), bin_edges_identity_batch.mean(axis=0)
    U_xz_transformed_hist[r], bin_edges_transformed[r] = U_xz_transformed_hist_batch.mean(axis=0), bin_edges_transformed_batch.mean(axis=0)
    U_xz_resampled_hist[r], bin_edges_resampled[r] = U_xz_resampled_hist_batch.mean(axis=0), bin_edges_resampled_batch.mean(axis=0)

for r in range(N_replicas):
    U_xz_bootstrap[r], _, Delta_U_xz_bootstrap[r] = bootstrap_observable(U_z, Tinvx[r], n_bootstrap, ref_bins, log_w_xz[r].cpu().numpy(), batch_size=None)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

fig_size = (10 * 0.393701, 6 * 0.393701)
fig = plt.figure(figsize = fig_size, dpi = 400)
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('text.latex', preamble=r"\usepackage{braket}")

plt.bar(mids, U_xz_reference_hist, alpha=0.5, width=bin_width, label=r"Reference", color="C0")
for r in range(N_replicas):
    if r == 0:
        plt.bar(mids, U_xz_resampled_hist[r], alpha=0.2, width=bin_width, label=r"Resampled", color="C3")
    else:
        plt.bar(mids, U_xz_resampled_hist[r], alpha=0.2, width=bin_width, color="C3")

plt.legend(frameon=False)

plt.xlabel(r"Energy(RU)")
plt.title(r"Energy $\mathrm{B} \to \mathrm{A}$")
plt.show()

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

fig_size = (10 * 0.393701, 6 * 0.393701)
fig = plt.figure(figsize = fig_size, dpi = 400)
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('text.latex', preamble=r"\usepackage{braket}")

plt.bar(mids, U_xz_reference_hist, alpha=0.5, width=bin_width, label=r"Reference", color="C0")
for r in range(N_replicas):
    if r == 0:
        plt.bar(mids, U_xz_bootstrap[r], alpha=0.2, width=bin_width, label=r"Bootstrapped", color="C3")
    else:
        plt.bar(mids, U_xz_bootstrap[r], alpha=0.2, width=bin_width, color="C3")

plt.legend(frameon=False)

plt.xlabel(r"Energy(RU)")
plt.title(r"Energy $\mathrm{A} \to \mathrm{B}$")
plt.show()

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

fig_size = (10 * 0.393701, 6 * 0.393701)
fig = plt.figure(figsize = fig_size, dpi = 400)
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('text.latex', preamble=r"\usepackage{braket}")

plt.bar(mids, U_xz_reference_hist, alpha=0.5, width=bin_width, label=r"Reference", color="C0", zorder=25)
plt.bar(mids, U_xz_identity_hist.mean(axis=0), alpha=0.5, width=bin_width, label=r"Identity", color="C1", zorder=25)
plt.bar(mids, U_xz_transformed_hist.mean(axis=0), alpha=0.5, width=bin_width, label=r"Transformed", color="C2", zorder=25)
Delta_U_xz_bootstrap = U_xz_bootstrap.std(axis=0, ddof=1)/np.sqrt(N_replicas)
plt.bar(mids, U_xz_bootstrap.mean(axis=0), color="C3", alpha=0.5, width=bin_width, yerr=Delta_U_xz_bootstrap, error_kw={'ecolor' : 'C3', 'alpha' : 0, 'lw' : 2.6}, label=r"Reweighted", zorder=25)
# Delta_U_xz_resampled = U_xz_resampled_hist.std(axis=0, ddof=1)
# plt.bar(mids, U_xz_resampled_hist.mean(axis=0), color="C3", alpha=0.5, width=bin_width, yerr=Delta_U_xz_resampled, error_kw={'ecolor' : 'C3', 'alpha' : 0.25, 'lw' : 4.01}, label=r"Resampled", zorder=25)

U_mean_reference = np.average(mids, weights=U_xz_reference_hist)
U_mean_bootstrap = np.average(mids, weights=U_xz_bootstrap.mean(axis=0))
U_mean_resampled = np.average(mids, weights=U_xz_resampled_hist.mean(axis=0))
plt.axvline(x = U_mean_reference, ymin = 0, c='C0', ls="--", lw=1, alpha=0.5, label=r"$\braket{U_\mathrm{A}}$ Reference")
plt.axvline(x = U_mean_bootstrap, ymin = 0, c='C3', ls="--", lw=1, alpha=0.5, label=r"$\Braket{U_\mathrm{A}\bigl(F^{-1}(x_{\mathrm{B}})\bigr)}$")
# plt.axvline(x = U_mean_resampled, ymin = 0.1, c='C4', ls="--", lw=0.5, alpha=0.5, label=r"$\Braket{U_\mathrm{A}\bigl(F^{-1}(x_{\mathrm{B}})\bigr)}$")

xlim = plt.xlim()
ylim = plt.ylim()

# plt.bar(U_mean_reference, height=ylim[1]+1, width=1, alpha=0.25, color='C0', zorder=0)
# plt.bar(U_mean_bootstrap, height=ylim[1]+1, width=1, alpha=0.25, color='C3', zorder=5)
# plt.bar(U_mean_resampled, height=ylim[1]+1, width=1, alpha=0.25, color='C4', zorder=5)

#get handles and labels
handles, labels = plt.gca().get_legend_handles_labels()

#specify order of items in legend
order = [2,3,4,5,0,1]

#add legend to plot
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order],frameon=False) 

# x = []
# y = []
# xlabels = []
# ylabels = []
# plt.xticks(x, xlabels)
# plt.yticks(y, ylabels)

# plt.legend(frameon=False) 

plt.xlim(xlim)
plt.ylim(0, ylim[1])
plt.xlabel(r"$U^*$")
plt.savefig(os.path.join(gendir, "energyB2A.png"))
# plt.title(r"Potential Energy Distribution $\mathrm{B} \to \mathrm{A}$")
plt.show()

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

fig_size = (10 * 0.393701, 6 * 0.393701)
fig = plt.figure(figsize = fig_size, dpi = 400)
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('text.latex', preamble=r"\usepackage{braket}")

linew = 0.2
bck_alpha = 0.4

plt.plot(mids, U_xz_reference_hist, label=r"Reference", lw=linew, color="C0")
plt.fill_between(mids, np.zeros_like(U_xz_reference_hist), U_xz_reference_hist, color="C0", alpha=bck_alpha)

plt.plot(mids, U_xz_identity_hist.mean(0), label=r"Untransformed", lw=linew, color="C1")
plt.fill_between(mids, np.zeros_like(U_xz_identity_hist.mean(0)), U_xz_identity_hist.mean(0), color="C1", alpha=bck_alpha)

plt.plot(mids, U_xz_transformed_hist.mean(0), label=r"Transformed", lw=linew, color="C2")
plt.fill_between(mids, np.zeros_like(U_xz_transformed_hist.mean(0)), U_xz_transformed_hist.mean(0), color="C2", alpha=bck_alpha)

xlim = plt.xlim()
ylim = plt.ylim()

density_mean = np.average(U_xz_resampled_hist, axis=0)
density_std = np.std(U_xz_resampled_hist, axis=0)

plt.plot(mids, density_mean, label=r"Resampled", lw=linew, color="C3")
plt.fill_between(mids, density_mean-density_std, density_mean+density_std, lw=0, color="C3", alpha=0.5)
plt.fill_between(mids, np.zeros_like(density_mean), density_mean, color="C3", alpha=bck_alpha)

plt.legend(frameon=False) 

# xlim = plt.xlim()
plt.ylim(ylim)
plt.xlabel(r"Energy (RU)")
plt.title(r"Potential Energy Distribution $\mathrm{B} \to \mathrm{A}$")
# plt.savefig(results_filepath)
plt.show()

## Radial Distribution Functions

In [ ]:
from tools.observables import rdf


n_bins = 100
cutoff = box_length_source/2
# Prior to System
RDF_r, RDF_xz_reference = rdf(wca_sample, n_particles=n_particles, dimensions=dimensions, box_length=box_length_pr, cutoff=cutoff, n_bins=n_bins, batch_size=None)

# RDF calculation
RDF_xz_identity = np.zeros((N_replicas, n_bins))
RDF_xz_transformed = np.zeros((N_replicas, n_bins))
RDF_xz_resampled = np.zeros((N_replicas, n_bins))
RDF_xz_reweighted = np.zeros((N_replicas, n_bins))

for r in range(N_replicas):
    _, RDF_xz_identity[r] = rdf(x[r]/scale, n_particles=n_particles, dimensions=dimensions, box_length=box_length_pr, cutoff=cutoff, n_bins=n_bins, batch_size=None)     
    _, RDF_xz_transformed[r] = rdf(Tinvx[r], n_particles=n_particles, dimensions=dimensions, box_length=box_length_pr, cutoff=cutoff, n_bins=n_bins, batch_size=None)
    _, RDF_xz_resampled[r] = rdf(Tinvx_resampled[r], n_particles=n_particles, dimensions=dimensions, box_length=box_length_pr, cutoff=cutoff, n_bins=n_bins, batch_size=None)
    _, RDF_xz_reweighted[r] = rdf(Tinvx[r], n_particles=n_particles, dimensions=dimensions, box_length=box_length_pr, cutoff=cutoff, n_bins=n_bins, batch_size=None, log_weights=log_w_xz[r])


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

fig_size = (10 * 0.393701, 6 * 0.393701)
fig = plt.figure(figsize = fig_size, dpi = 400)
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('text.latex', preamble=r"\usepackage{braket}")

plt.axhline(y = 1, c='k', ls="--", lw=1, alpha=0.25)

plt.plot(RDF_r, RDF_xz_identity.mean(0), label=r"Identity", ls='--', color='C1')
plt.plot(RDF_r, RDF_xz_transformed.mean(0), label=r"Transformed", ls='-.', color='C2')
# plt.plot(RDF_r, RDF_xz_resampled.mean(0), label=r"Resampled", ls='-', color='C3')
plt.plot(RDF_r, RDF_xz_reweighted.mean(0), label=r"Reweighted", ls='-', color='C3')
plt.plot(RDF_r, RDF_xz_reference, label=r"Reference", ls=':', color='C0')
  
#get handles and labels
handles, labels = plt.gca().get_legend_handles_labels()

#specify order of items in legend
order = [3,0,1,2]

#add legend to plot
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order],frameon=False) 

# plt.legend(frameon=False)

plt.xlabel(r"$r^*$")
plt.ylabel(r"$g(r^*)$")
# plt.title(r"Radial Distribution Function $\mathrm{B}\to \mathrm{A}$")
# plt.savefig(results_filepath)
plt.savefig(os.path.join(gendir, "rdfB2A.png"))
plt.show()

## RESS

In [ ]:
rep_xz, ress_id_xz, ress_xz = np.loadtxt(os.path.join(gendir, "ress_xz.out"), unpack=True, usecols=(0,1,2))
n_rep = rep_zx.shape[0]

In [ ]:
print("x->z")
print(f"ress id :{ress_id_xz.mean()} +- {ress_id_xz.std(ddof=1)/np.sqrt(n_rep)}\tDelta = {ress_id_xz.std(ddof=1)/np.sqrt(n_rep)/ress_id_xz.mean()}")
print(f"ress :{ress_xz.mean()} +- {ress_xz.std(ddof=1)/np.sqrt(n_rep)}\tDelta = {ress_xz.std(ddof=1)/np.sqrt(n_rep)/ress_xz.mean()}")
print(f"ratio : {ress_xz.mean()/ress_id_xz.mean()}")